In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,4975
2,Huelva,Confirmados PDIA 14 días,2321
3,Huelva,Tasa PDIA 14 días,"454,4359883542212"
4,Huelva,Confirmados PDIA 7 días,1242
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,38
629,Municipio de Huelva sin especificar,Total Confirmados,99
630,Municipio de Huelva sin especificar,Curados,20


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  4975.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  1314.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 459 personas en los últimos 7 días 

Un positivo PCR cada 228 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,4975.0,1242.0,2321.0,510743.0,243.175139,454.435988,838.0
Huelva-Costa,2484.0,592.0,1138.0,288115.0,205.473509,394.981171,401.0
Condado-Campiña,1643.0,450.0,782.0,155057.0,290.215856,504.330665,301.0
Huelva (capital),1314.0,313.0,630.0,143663.0,217.870990,438.526273,223.0
Sierra de Huelva-Andévalo Central,749.0,162.0,334.0,67571.0,239.747821,494.294890,102.0
Lepe,378.0,103.0,178.0,27431.0,375.487587,648.900879,61.0
Palma del Condado (La),243.0,63.0,115.0,10761.0,585.447449,1068.673915,53.0
Almonte,272.0,87.0,164.0,24191.0,359.637882,677.938076,50.0
Gibraleón,205.0,84.0,145.0,12607.0,666.296502,1150.154676,49.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Escacena del Campo,76.0,34.0,56.0,2284.0,1488.616462,2451.838879,18.0
Villarrasa,66.0,16.0,32.0,2176.0,735.294118,1470.588235,9.0
Gibraleón,205.0,84.0,145.0,12607.0,666.296502,1150.154676,49.0
Palma del Condado (La),243.0,63.0,115.0,10761.0,585.447449,1068.673915,53.0
Aracena,148.0,57.0,77.0,8107.0,703.096090,949.796472,43.0
Calañas,29.0,6.0,25.0,2791.0,214.976711,895.736295,3.0
Valverde del Camino,146.0,40.0,100.0,12820.0,312.012480,780.031201,26.0
Cortegana,117.0,5.0,36.0,4666.0,107.158165,771.538791,2.0
Almonaster la Real,27.0,6.0,14.0,1819.0,329.851567,769.653656,4.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cortegana,117.0,5.0,36.0,4666.0,107.158165,771.538791,2.0,0.138889
Calañas,29.0,6.0,25.0,2791.0,214.976711,895.736295,3.0,0.240000
Cerro de Andévalo (El),28.0,1.0,4.0,2364.0,42.301184,169.204738,1.0,0.250000
San Bartolomé de la Torre,10.0,1.0,4.0,3736.0,26.766595,107.066381,1.0,0.250000
Granado (El),34.0,1.0,3.0,516.0,193.798450,581.395349,0.0,0.333333
Minas de Riotinto,24.0,1.0,3.0,3848.0,25.987526,77.962578,0.0,0.333333
Cartaya,115.0,23.0,63.0,19974.0,115.149695,315.410033,17.0,0.365079
Valverde del Camino,146.0,40.0,100.0,12820.0,312.012480,780.031201,26.0,0.400000
Bollullos Par del Condado,160.0,14.0,34.0,14272.0,98.094170,238.228700,5.0,0.411765
